<h1 style="text-align:center;"><b>Laboratorio 2</b></h1>
<h3 style="text-align:center;">Marcos Díaz (221102), Daniel Machic (22118), Maria Jose Ramírez (221051)</h3>

**Github**: https://github.com/mac2218/Seminario-Lab02

# Librerias

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# **Ejercicio 1**

In [ ]:
# Probabilidades conjuntas P(X, Y)
joint = np.array([
    [0.09, 0.16, 0.17, 0.05],  # Y=0
    [0.15, 0.07, 0.10, 0.21]   # Y=1
])


px = joint.sum(axis=0)


posterior = joint / px


decision = np.argmax(posterior, axis=0)

# Error de Bayes
error = sum(px[i] * np.min(posterior[:, i]) for i in range(len(px)))

print("Posteriores P(Y|X):\n", posterior)
print("\nRegla óptima (0=Y0, 1=Y1):\n", decision)
print("\nError de Bayes:", error)


Posteriores P(Y|X):
 [[0.375      0.69565217 0.62962963 0.19230769]
 [0.625      0.30434783 0.37037037 0.80769231]]

Regla óptima (0=Y0, 1=Y1):
 [1 0 0 1]

Error de Bayes: 0.31


## Clasificador Bayesiano Óptimo

### 1. Probabilidades posteriores

A partir de la tabla de probabilidades conjuntas \( P(X,Y) \), calculamos:
$P(Y \mid X=x) = \frac{P(X=x, Y)}{P(X=x)}$

Las probabilidades posteriores obtenidas son:

$
P(Y \mid X)$ =
\begin{bmatrix}
0.375 & 0.6957 & 0.6296 & 0.1923 \\
0.625 & 0.3043 & 0.3704 & 0.8077
\end{bmatrix}


donde la primera fila corresponde a \( Y=0 \) y la segunda a \( Y=1 \).

---

### 2. Regla de clasificación óptima

El clasificador Bayesiano asigna la clase con mayor probabilidad posterior:

$hat{y}(x) = \arg\max_{y \in \{0,1\}} P(Y=y \mid X=x)$

Por lo tanto:

$
\hat{y}(1)=1, \quad
\hat{y}(2)=0, \quad
\hat{y}(3)=0, \quad
\hat{y}(4)=1
$

---

### 3. Error de Bayes

El error mínimo de clasificación está dado por:

$
P_e = \sum_{x} P(X=x)\min \{P(Y=0 \mid X=x), P(Y=1 \mid X=x)\}
$
Sustituyendo valores:

$
P_e = 0.31
$

Por lo tanto, el error del clasificador Bayesiano óptimo es:

$
\boxed{31\%}
$


# Ejercicio 2

In [ ]:
# Punto de decisión Bayesiano
x_star = np.log(3) / 2

print("Punto de decisión (x*):", x_star)

# Regla de clasificación

def clasificador_bayes(x):
    """
    Regla Bayesiana óptima:
    Clasificar como Y=1 si x < x*
    Clasificar como Y=0 si x >= x*
    """
    if x < x_star:
        return 1
    else:
        return 0

print("\nRegla de clasificación:")
print("Si x <", x_star, "→ Y = 1")
print("Si x >=", x_star, "→ Y = 0")

# Regiones de clasificación

print("\nRegiones de clasificación:")
print("R1 = [0,", x_star, ")")
print("R0 = [", x_star, ", ∞)")

# Cálculo del error
# Error = 0.5 P(X∈R1|Y=0) + 0.5 P(X∈R0|Y=1)

error = 0.5 * (1 - np.exp(-x_star)) + 0.5 * np.exp(-3 * x_star)

print("\nError del clasificador:", error)

Punto de decisión (x*): 0.5493061443340549

Regla de clasificación:
Si x < 0.5493061443340549 → Y = 1
Si x >= 0.5493061443340549 → Y = 0

Regiones de clasificación:
R1 = [0, 0.5493061443340549 )
R0 = [ 0.5493061443340549 , ∞)

Error del clasificador: 0.3075499102701248


El clasificador bayesiano óptimo para las distribuciones exponenciales consideradas establece un punto de decisión en $x^* \approx 0.5493$. Esto significa que valores pequeños de la variable observada se asocian más probablemente a la clase $Y=1$ (distribución con parámetro 3), mientras que valores mayores se asignan a la clase $Y=0$ (distribución con parámetro 1). La razón es que la distribución con parámetro mayor decrece más rápidamente, concentrando mayor probabilidad en valores cercanos a cero. El error del clasificador es aproximadamente $0.308$, lo que indica que, aun siendo óptimo en el sentido bayesiano, existe un solapamiento considerable entre ambas distribuciones, generando una probabilidad no despreciable de clasificación incorrecta.

# Ejercicio 3

In [ ]:
# Datos
data = {
    "Outlook": ["Rainy","Rainy","Overcast","Sunny","Sunny","Sunny","Overcast","Rainy",
                "Rainy","Sunny","Rainy","Overcast","Overcast","Sunny"],
    "Temperature": ["Hot","Hot","Hot","Mild","Cool","Cool","Cool","Mild",
                    "Cool","Mild","Mild","Mild","Hot","Mild"],
    "Humidity": ["High","High","High","High","Normal","Normal","Normal","High",
                 "Normal","Normal","Normal","High","Normal","High"],
    "Windy": [False,True,False,False,False,True,True,False,
              False,False,True,True,False,True],
    "Play": ["No","No","Yes","Yes","Yes","No","Yes","No",
             "Yes","Yes","Yes","Yes","Yes","No"]
}

df = pd.DataFrame(data)

# Codificar variables categóricas
df_encoded = df.copy()

encoders = {}
for col in df.columns:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df[col])
    encoders[col] = le

X = df_encoded.drop("Play", axis=1)
y = df_encoded["Play"]

## Naive Bayes

In [7]:
from sklearn.naive_bayes import CategoricalNB

model_nb = CategoricalNB()
model_nb.fit(X, y)

# Casos a clasificar
test = pd.DataFrame({
    "Outlook": ["Rainy", "Sunny"],
    "Temperature": ["Hot", "Hot"],
    "Humidity": ["High", "Normal"],
    "Windy": [False, False]
})

# Codificar test usando mismos encoders
test_encoded = test.copy()
for col in test.columns:
    test_encoded[col] = encoders[col].transform(test[col])

pred_nb = model_nb.predict(test_encoded)

print("Predicciones Naive Bayes:", encoders["Play"].inverse_transform(pred_nb))

Predicciones Naive Bayes: ['No' 'Yes']


## KNN

In [8]:
from sklearn.neighbors import KNeighborsClassifier

model_knn = KNeighborsClassifier(n_neighbors=3)
model_knn.fit(X, y)

pred_knn = model_knn.predict(test_encoded)

print("Predicciones KNN:", encoders["Play"].inverse_transform(pred_knn))

Predicciones KNN: ['No' 'Yes']


## Métricas de desempeño

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Predicciones sobre el entrenamiento (dataset pequeño)
y_pred_nb = model_nb.predict(X)
y_pred_knn = model_knn.predict(X)

print("=== NAIVE BAYES ===")
print("Accuracy:", accuracy_score(y, y_pred_nb))
print("Precision:", precision_score(y, y_pred_nb))
print("Recall:", recall_score(y, y_pred_nb))
print("F1 Score:", f1_score(y, y_pred_nb))
print("ROC AUC:", roc_auc_score(y, model_nb.predict_proba(X)[:,1]))

print("\n=== KNN ===")
print("Accuracy:", accuracy_score(y, y_pred_knn))
print("Precision:", precision_score(y, y_pred_knn))
print("Recall:", recall_score(y, y_pred_knn))
print("F1 Score:", f1_score(y, y_pred_knn))

=== NAIVE BAYES ===
Accuracy: 0.9285714285714286
Precision: 0.9
Recall: 1.0
F1 Score: 0.9473684210526315
ROC AUC: 0.9222222222222223

=== KNN ===
Accuracy: 0.8571428571428571
Precision: 0.8888888888888888
Recall: 0.8888888888888888
F1 Score: 0.8888888888888888


En el conjunto de datos “Play Golf”, ambos clasificadores producen las mismas predicciones para los casos solicitados. Sin embargo, al evaluar las métricas de desempeño, Naive Bayes presenta mejores resultados generales, con mayor Accuracy, F1 Score y ROC AUC. En particular, alcanza un Recall de 1.0, lo que indica que identifica correctamente todas las instancias positivas. Esto se debe a que Naive Bayes es especialmente adecuado para variables categóricas y conjuntos de datos pequeños, mientras que KNN puede verse afectado por la codificación numérica de variables categóricas y la elección del parámetro k. Por tanto, en este problema, Naive Bayes muestra un desempeño superior.

## **¿Cómo construir un clasificador bueno con tan pocos datos?**

Cuando se dispone de un conjunto de datos pequeño, como en este caso (14 observaciones), es fundamental utilizar modelos simples que tengan baja varianza y menor riesgo de sobreajuste. Algoritmos como Naive Bayes o Regresión Logística suelen ser más adecuados que modelos complejos, ya que requieren menos datos para estimar sus parámetros de manera estable.

Además, es importante aplicar técnicas como validación cruzada para obtener una estimación más confiable del desempeño, dado que separar en entrenamiento y prueba puede reducir demasiado el tamaño efectivo del conjunto de datos. También resulta clave incorporar conocimiento previo del problema y realizar una correcta codificación de variables. En general, con pocos datos, la simplicidad del modelo y la correcta validación son más importantes que la complejidad del algoritmo.